In [1]:
import numpy as np
import pandas as pd
from sklearn.metrics.pairwise import cosine_similarity, cosine_distances
from sklearn.feature_extraction.text import CountVectorizer
import ast
import warnings

warnings.filterwarnings('ignore') 

pd.set_option("display.max_columns", 100) # ‘None’ value means unlimited.

THEMES = {
 'amitie': 373,
 'amour': 77,
 'autobiographie': 45,
 'aventure': 33,
 'bande-dessinee': 18,
 'biographie': 31,
 'cinema': 89,
 'classique': 28,
 'comedie-romantique': 3788,
 'comics': 142,
 'drogue': 863,
 'dystopie': 879,
 'emotion': 4100,
 'enquetes': 3988,
 'entretiens': 708,
 'essai': 13,
 'famille': 290,
 'fantastique': 7,
 'fantasy': 4,
 'geographie': 187,
 'guerre': 91,
 'humour': 15,
 'humour-noir': 621,
 'jeunesse': 14,
 'journalisme': 975,
 'litterature-americaine': 9,
 'litterature-asiatique': 1064,
 'litterature-francaise': 3,
 'manga': 12,
 'musique': 44,
 'nouvelles': 23,
 'peur': 430,
 'poesie': 25,
 'politique': 54,
 'psychologie': 65,
 'racisme': 906,
 'recit-de-voyage': 448,
 'religion': 26,
 'reportage': 3488,
 'reseaux-sociaux': 26109,
 'roman': 1,
 'roman-fantastique': 912,
 'roman-noir': 136,
 'romans-policiers-et-polars': 63883,
 'science-fiction': 6,
 'sentiments': 1770,
 'serie': 788,
 'theatre': 21,
 'thriller': 11,
 'thriller-psychologique': 1073,
 'tragedie': 601,
 'western': 533
}



# Merging files and datasets

In [2]:
def merge_data_files(books_file_list:list, books_meta_file=None, books_senti_file=None, books_users_file=None):
    df_books = None

    # fusion of mains books data files with comments
    for filename in books_file_list:
        df_books_temp = pd.read_json(filename, lines=True)
        if df_books is None:
            df_books = df_books_temp
        else: df_books = pd.concat([df_books, df_books_temp])

    df_books = df_books.drop(['tags'],axis=1)

    # genre du profil per user_id
    if books_users_file is not None:
        df_users = pd.read_json(books_users_file, lines=True)
        df_books = df_books.merge(df_users, on='user_id', how='left')
    df_books = df_books.fillna('')

    # join to the meta data books file
    if books_meta_file is not None:
        df_meta = pd.read_json(books_meta_file, lines=True)
        df_meta = df_meta.drop(['book_nb_comm', 'title', 'name', 'surname','img_url','book_date'],axis=1)
        df_books = df_books.merge(df_meta, on='book_id', how='inner')

    # join sentiments file
    if books_senti_file is not None:
        df_senti = pd.read_json(books_senti_file, lines=True)
        df_senti = df_senti.drop(['title'],axis=1)
        df_books = df_books.merge(df_senti, on='book_id', how='left')
    df_books = df_books.fillna(0)

    return df_books

In [3]:
df_comm = merge_data_files(
    books_file_list = ['../output/books-julien.json','../output/books-rebecca.json'],
    books_meta_file = '../output/books-meta-data.json',
    books_senti_file = '../output/vecteurs_sentiments.json',
    books_users_file = '../output/users-data.json'
    )

In [6]:
# df_comm['year'] = df_comm['book_date'].str.extract(r'\b(\d{4})\b').replace('1900','').replace('3889',)
# df_comm['year'].unique()

In [5]:
df_found = df_books.loc[df_books['title'].str.contains('vautours', case=False)].reset_index()
title = df_found.loc[0,'title']
title

NameError: name 'df_books' is not defined

In [7]:
# group by per book
columns_senti = [col for col in df_comm.columns if col.startswith('sen_')]
columns_book = ['book_id', 'book_url', 'book_nb_comm', 'title', 'name', 'surname',
       'tags', 'img_url', 'book_rating_count', 'book_rating_value',
       'book_author_url', 'book_editor', 'book_pages', *columns_senti]

def reduce_comm_to_books(df):
    return df.copy().groupby(columns_book, as_index=False).count().loc[:,columns_book]

In [8]:
df_books = reduce_comm_to_books(df_comm)

We have now 2 dataset : df_comm and df_books with 2 differents aggregate levels

# Preprocessing df_books

In [9]:
# on garde que les tags avec le nom dans filter_list ou si la valeur est supérieur à filter_force_min. ca permet de rétirer les tags rares et peu importants
def tags_to_cols(df, col_name, filter_list=None, filter_force_min=24):
    df1 = df.copy()

    for index,row in df.iterrows():
        tags_as_string = row[col_name]
        tags = ast.literal_eval(tags_as_string)

        for tag in tags:
            if filter_list is not None and not tag[0].strip() in filter_list and tag[1] < filter_force_min:
                continue
            tag_name = 'tag_'+tag[0].strip().replace(' ','_').lower()
            df1.loc[df1.index == index, tag_name] = tag[1]

    df1 = df1.fillna(0)
    return df1

In [10]:
df_books = tags_to_cols(df_books, col_name='tags', filter_list=list(THEMES.keys()), filter_force_min=24)

In [11]:
df_books.to_json('../output/final/data-books.json',lines=True,orient='records')
df_comm.to_json('../output/final/data-comm.json',lines=True,orient='records')

In [25]:
df_jaccard = pd.read_csv('../output/jacsim.csv')
# df_jaccard = df_jaccard.rename(columns = {'Unnamed: 0':'book_id'})
# df_jaccard = df_jaccard.set_index('book_id')

# df1 = df_jaccard.query('book_id == 1390851').transpose().dropna().squeeze()
# df2 = df_jaccard.loc[:,'1390851'].dropna()

# pd.concat([df1,df2], axis=0)

In [13]:
df_jaccard

,1379642,466996,123961,338279,1300939,1079315,832220,56385,676627,698482,1127856,650958,1390851,830641,717829,5742,1186100,1224013,395220,903004,494737,1261456,40426,1000388,1358497,944573,766685,965802,17544,1185527,934827,1231250,1035039,574645,431764,1295258,1310212,124380,987659,335559,6999,14022,799846,220453,45277,132908,1299126,797175,1385832,471907,...,901109,7643,826777,808398,159116,556212,1192722,809728,1068833,1252905,766702,6532,1277512,201751,1241904,1158955,4397,23208,1270461,1316806,10374,2287,991495,1322555,1003722,1720,1282794,1353562,127445,886136,1308933,4696,916012,950395,534643,501077,1232499,6066,983150,357202,20799,812075,895066,22977,40020,1000176,794245,225627,1379634,81141
book_id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
1379642,1.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
466996,0.791045,1.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
123961,0.742857,0.928571,1.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
338279,0.690141,0.859649,0.862069,1.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1300939,0.728571,0.877193,0.847458,0.844828,1.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1000176,0.728571,0.910714,0.879310,0.877193,0.862069,0.731343,0.883333,0.864407,0.830508,0.877193,0.822581,0.796875,0.877193,0.864407,0.852459,0.910714,0.704225,0.460177,0.838710,0.892857,0.827586,0.862069,0.896552,0.881356,0.833333,0.757576,0.847458,0.864407,0.803279,0.816667,0.913793,0.604938,0.722222,0.910714,0.866667,0.510417,0.504950,0.803279,0.838710,0.945455,0.833333,0.910714,0.864407,0.875000,0.875000,0.844828,0.671233,0.758621,0.695652,0.816667,...,0.510000,0.910714,0.850000,0.896552,0.850000,0.862069,0.807018,0.896552,0.810345,0.847458,0.847458,0.912281,0.850000,0.857143,0.635135,0.879310,0.784615,0.815385,0.838710,0.657895,

In [14]:
df1 = df_jaccard.query('book_id == 1390851').transpose().dropna().squeeze()
df1

1379642    0.764706
466996     0.962963
123961     0.928571
338279     0.859649
1300939    0.844828
1079315    0.796875
832220     0.836066
56385      0.912281
676627     0.877193
698482     0.962963
1127856    0.866667
650958     0.838710
1390851    1.000000
Name: 1390851, dtype: float64

In [15]:
df1 = df_jaccard.query('book_id == 1390851').transpose().dropna().squeeze()
df1

1379642    0.764706
466996     0.962963
123961     0.928571
338279     0.859649
1300939    0.844828
1079315    0.796875
832220     0.836066
56385      0.912281
676627     0.877193
698482     0.962963
1127856    0.866667
650958     0.838710
1390851    1.000000
Name: 1390851, dtype: float64

In [16]:
df2 = df_jaccard.loc[:,'1390851'].dropna().squeeze()

In [17]:
pd.concat([df1,df2], axis=0)

1379642    0.764706
466996     0.962963
123961     0.928571
338279     0.859649
1300939    0.844828
             ...   
1000176    0.877193
794245     0.962963
225627     0.875000
1379634    0.912281
81141      0.962963
Length: 1501, dtype: float64

In [18]:
df_jaccard.loc[:,'1390851']

book_id
1379642         NaN
466996          NaN
123961          NaN
338279          NaN
1300939         NaN
             ...   
1000176    0.877193
794245     0.962963
225627     0.875000
1379634    0.912281
81141      0.962963
Name: 1390851, Length: 1500, dtype: float64

# Simple Model

In [142]:
from sklearn.preprocessing import StandardScaler, MinMaxScaler

# todo : index et book_id, classer les scores par colonnes dans un dataframe avec en clé le book_id

class BookReco:
    def __init__(self):
        self.data = None
        self.vectors = []
        self.scalars = []
        self.predict_scores = None
        self.weights = {}
        self.df_jaccard = None
        self.data_score = None

    def add_vector(self, col_prefix, weight=1):
        self.weights = {**self.weights, col_prefix: weight}
        self.vectors.append({'col_name': col_prefix})

    def add_jaccard(self, df, weight=1):
        self.weights = {**self.weights, 'jaccard': weight}
        self.df_jaccard = df

    def add_scalar(self, col_name, weight=1):
        self.weights = {**self.weights, col_name: weight}
        self.scalars.append({'col_name': col_name})

    def __get_cosine_similarity(self, prefix):
        data_temp = self.data.filter(regex=f'^{prefix}',axis=1)
        data_temp = data_temp.fillna(0)
        vec = MinMaxScaler().fit_transform(data_temp)
        return cosine_similarity(vec)

    def fit(self, data):
        self.data = data

        for i,vector in enumerate(self.vectors):
            feats_cs = self.__get_cosine_similarity(vector['col_name'])
            self.vectors[i] = {**vector, 'cosine_similar': feats_cs}

        for i,scalar in enumerate(self.scalars):
            X = self.data.loc[:,[scalar['col_name']]]
            X = MinMaxScaler().fit_transform(X)
            self.scalars[i] = {**scalar, 'scaled': X.reshape(-1)}

    def set_weight(self, weights:dict):
        self.weights = weights

    def __get_jaccard_score(self, book_id):
        df_jaccard = self.df_jaccard.copy()
        df_jaccard = df_jaccard.rename(columns = {'Unnamed: 0':'book_id'})
        df_jaccard = df_jaccard.set_index('book_id')

        try:
            df1 = df_jaccard.query('book_id == @book_id').transpose().dropna().squeeze()
            df2 = df_jaccard.loc[:, str(book_id)].dropna()
        except:
            print('Error Jaccard in __get_jaccard_score')
            return None

        return pd.concat([df1,df2], axis=0)

    def predict(self, book_id):

        try:
            index_book = self.data.query('book_id == @book_id').index.values.astype(int)[0]
        except:
            print(f"Can't find book_id: {book_id} in the dataset")
            return None
        
        # init data_score to empty
        self.data_score = self.data[['book_id']]

        # get all scores, apply weight
        weight_sum = 0 # to normalize at the end, like a mean
        for vector in self.vectors:
            score = vector['cosine_similar'][index_book]
            weight = self.weights[vector['col_name']]
            weight_sum += weight
            display(pd.Series(score*weight, name=vector['col_name']))
            self.data_score = pd.concat([self.data_score, pd.Series(score*weight, name=vector['col_name'])], axis=1)
        
        for scalar in self.scalars:
            weight = self.weights[scalar['col_name']]
            weight_sum += weight
            self.data_score = pd.concat([self.data_score, pd.Series(scalar['scaled']*weight, name=scalar['col_name'])], axis=1)

        self.data_score = self.data_score.set_index('book_id')

        
        if self.df_jaccard is not None:
            score = self.__get_jaccard_score(book_id) 

            if score is not None:
                score = score * self.weights['jaccard']
                weight_sum += self.weights['jaccard']
                self.data_score = self.data_score.merge(pd.Series(score, name='jaccard'), how='left', left_index=True, right_index=True)

        if weight_sum == 0:
            weight_sum == 1

        self.data_score =  self.data_score.sum(axis=1) / weight_sum

        return self.data.merge(pd.Series(self.data_score, name='score'), how='left', left_on='book_id', right_index=True)

    def format_tojson(self, scores, max_books):

        output = []
        num = 0

        scores = scores.sort_values('score', ascending=False)

        for _, book in scores.iterrows():
            if num == 0: # we ignore the first one which is the book with whhch we want predictions
                num += 1
                continue

            output.append({
                'title' : book['title'],
                'url' : book['book_url'],
                'image' : book['img_url'],
                'author' : book['surname']+' '+book['name'],
                'author_url' : book['book_author_url'],
                'book_rating_value' : book['book_rating_value'],
                'book_rating_count' : book['book_rating_count'],
                'score' : str(book['score']),
            })
            if num == max_books:
                break
            num += 1
        return output


In [145]:
# import modelreco

br = BookReco()
br.add_vector('tag_', weight=0)
br.add_vector('sen_', weight=0)
br.add_jaccard(df_jaccard, weight=1)
br.add_scalar('book_rating_value', weight=0)
br.add_scalar('book_nb_comm', weight=0)
br.fit(df_books)

br.set_weight({'tag_': 0.5, 'sen_': 10, 'jaccard': 1, 'book_rating_value': 0, 'book_nb_comm': 0})
scores = br.predict(123961) # De Cape et de Crocs, tome 2 : Pavillon noir !
#br.format_tojson(scores, max_books=5)

0       0.000000
1       0.135070
2       0.170567
3       0.000000
4       0.051112
          ...   
4561    0.243702
4562    0.200603
4563    0.000000
4564    0.000000
4565    0.000000
Name: tag_, Length: 4566, dtype: float64

0       0.000000
1       0.000000
2       0.000000
3       0.000000
4       0.000000
          ...   
4561    0.000000
4562    0.000000
4563    0.000000
4564    8.443766
4565    0.000000
Name: sen_, Length: 4566, dtype: float64

Error Jaccard in __get_jaccard_score


,tag_,sen_,book_rating_value,book_nb_comm
book_id,,,,
1497,0.000000,0.000000,0.0,0.0
1499,0.135070,0.000000,0.0,0.0
1508,0.170567,0.000000,0.0,0.0
1523,0.000000,0.000000,0.0,0.0
1526,0.051112,0.000000,0.0,0.0
...,...,...,...,...
1462096,0.243702,0.000000,0.0,0.0
1462133,0.200603,0.000000,0.0,0.0
1466000,0.000000,0.000000,0.0,0.0


In [141]:
br.set_weight({'tag_': 0.5, 'sen_': 10, 'jaccard': 1, 'book_rating_value': 0, 'book_nb_comm': 0})
scores = br.predict(1829) # De Cape et de Crocs, tome 2 : Pavillon noir !
br.format_tojson(scores, max_books=5)

0       0.000000
1       0.135070
2       0.170567
3       0.000000
4       0.051112
          ...   
4561    0.243702
4562    0.200603
4563    0.000000
4564    0.000000
4565    0.000000
Name: tag_, Length: 4566, dtype: float64

0       0.000000
1       0.000000
2       0.000000
3       0.000000
4       0.000000
          ...   
4561    0.000000
4562    0.000000
4563    0.000000
4564    8.443766
4565    0.000000
Name: sen_, Length: 4566, dtype: float64

Error Jaccard in __get_jaccard_score


[{'title': "L'Odyssée",
  'url': 'https://www.babelio.com/livres/Homere-LOdyssee/4228',
  'image': 'https://m.media-amazon.com/images/I/412vlscbdtL._SX195_.jpg',
  'author': 'Homère ',
  'author_url': '/auteur/-Homere/3668',
  'book_rating_value': 3.92,
  'book_rating_count': 2799,
  'score': '0.9270478725969363'},
 {'title': 'La guerre des mondes',
  'url': 'https://www.babelio.com/livres/Wells-La-guerre-des-mondes/3404',
  'image': '/couv/CVT_La-guerre-des-mondes_7069.jpg',
  'author': 'Wells H.G.',
  'author_url': '/auteur/HG-Wells/26217',
  'book_rating_value': 3.84,
  'book_rating_count': 2864,
  'score': '0.9045683433278255'},
 {'title': 'Sa majesté des mouches',
  'url': 'https://www.babelio.com/livres/Golding-Sa-majeste-des-mouches/1569',
  'image': '/couv/CVT_Sa-majeste-des-mouches_6047.jpg',
  'author': 'Golding William',
  'author_url': '/auteur/William-Golding/2025',
  'book_rating_value': 3.76,
  'book_rating_count': 3212,
  'score': '0.8884725818373831'},
 {'title': 'Runn

In [ ]:
import pickle
import dill

with open('data/model-reco.obj', "wb") as f:
    dill.dump(br, f)

# with open('data/model-reco.obj', 'wb') as f:
#     pickle.dump(br, f, pickle.HIGHEST_PROTOCOL)